In [ ]:
import arcpy
import pandas as pd

gdb = arcpy.env.workspace
#summarise REs

if arcpy.Exists("Processing"):
    arcpy.Delete_management("Processing")
    
arcpy.management.CreateFeatureDataset(gdb, 'Processing', arcpy.Describe(f'{gdb}/Input').spatialReference.name)

# copy processing dataset
arcpy.management.CopyFeatures('Input/regionalEcosystemsMerged', 'Processing/reSum')

# unique res
res = ['12.1.1', '12.1.2', '12.1.3', '12.1.3f', '12.11.1', '12.11.10', '12.11.11', '12.11.14', '12.11.18', '12.11.18a', '12.11.19', '12.11.2', '12.11.24', '12.11.25', '12.11.27', '12.11.3', '12.11.3a', '12.11.3b', '12.11.5', '12.11.9', '12.11.9x1', '12.12.1', '12.12.12', '12.12.13', '12.12.14', '12.12.15', '12.12.15a', '12.12.15b', '12.12.16', '12.12.2', '12.12.23', '12.12.28', '12.12.3', '12.12.3a', '12.12.6', '12.12.7', '12.12.8', '12.2.12', '12.2.14', '12.2.15', '12.2.15a', '12.2.5', '12.2.6', '12.2.7', '12.2.9', '12.3.11', '12.3.13', '12.3.14', '12.3.14a', '12.3.16', '12.3.1a', '12.3.2', '12.3.20', '12.3.3', '12.3.3d', '12.3.4', '12.3.4a', '12.3.5', '12.3.56', '12.3.6', '12.3.6c', '12.3.7', '12.3.7c', '12.3.7d', '12.3.8', '12.5.2a', '12.5.3', '12.5.3a', '12.5.3c', '12.5.6c', '12.5.7', '12.5.7b', '12.8.14', '12.8.20', '12.8.25', '12.8.3', '12.8.8', '12.9-10.14', '12.9-10.16', '12.9-10.17', '12.9-10.17a', '12.9-10.19a', '12.9-10.3', '12.9-10.4']

# add new refined regional ecosystem columns
colsre = ['rre1', 'rre2', 'rre3', 'rre_pc1', 'rre_pc2', 'rre_pc3']

for col in colsre:
    if col not in [field.name for field in arcpy.ListFields('reSum')]:
        typ = 'DOUBLE' if 'pc' in col else 'TEXT'
        arcpy.management.AddField('reSum', col, typ)

# split refined re and pc across columns
arcpy.management.SelectLayerByAttribute('reSum', 'NEW_SELECTION', "refined_re <> ''")
# re
cols = ['refined_re', 'rre1', 'rre2', 'rre3']
with arcpy.da.UpdateCursor('reSum', cols) as cursor:
    for row in cursor:
        strsplit = row[0].split('/')
        for i in range(1,4):
            try:
                row[i] = strsplit[i-1]
            except IndexError:
                continue
        cursor.updateRow(row)

# percentages
cols = ['refined_percent', 'rre_pc1', 'rre_pc2', 'rre_pc3']
with arcpy.da.UpdateCursor('reSum', cols) as cursor:
    for row in cursor:
        try:
            strsplit = row[0].split('/')
        except AttributeError:
            continue
        for i in range(1,4):
            try:
                row[i] = int(strsplit[i-1])
            except IndexError:
                continue
        cursor.updateRow(row)
        
# some incorrect percentages exist. Ensure that if there is only one RE, pc1 = 100.
with arcpy.da.UpdateCursor('reSum', ['refined_re', 'rre_pc1']) as cursor:
    if row[0] != '' and '/' not in row[0]:
        row[1] = 100

arcpy.management.SelectLayerByAttribute('reSum', "CLEAR_SELECTION")

In [ ]:
# build RE dictionary that we will store summed areas to.
reDict = {key: 0 for key in res}

# first, iterate through refined RE polygons
arcpy.management.SelectLayerByAttribute('reSum', 'NEW_SELECTION', "refined_re <> '' AND refined_re <> 'Planted E. grandis open forest'")
cols = ['rre1', 'rre2', 'rre3', 'rre_pc1', 'rre_pc2', 'rre_pc3', 'SHAPE_Area']
with arcpy.da.SearchCursor('reSum', cols) as cursor:
    for row in cursor:
        # build dictionary
        areaDict = {row[0]:row[3],
                   row[1]:row[4],
                   row[2]:row[5]}
        # get area per rre column
        for re, rePc in areaDict.items():
            if re is not None:
                try:
                    area = ((rePc / 100) * row[6]) / 10000
                except TypeError:
                    area = 0
                # add area to reDictionary
                try:
                    reDict[re] += area
                except KeyError:
                    continue

reDict
                
# now iterate through the rest of the polygons
arcpy.management.SelectLayerByAttribute('reSum', 'NEW_SELECTION', "refined_re = ''")

cols = ['RE1', 'RE2', 'RE3', 'RE4', 'RE5', 'PC1', 'PC2', 'PC3', 'PC4', 'PC5', 'SHAPE_Area']
with arcpy.da.SearchCursor('reSum', cols) as cursor:
    for row in cursor:
        # build dictionary
        areaDict = {row[0]:row[5],
                   row[1]:row[6],
                   row[2]:row[7],
                   row[3]:row[8],
                   row[4]:row[9]}
        # get area per re column
        for re, rePc in areaDict.items():
            if re is not None:
                try:
                    area = ((rePc / 100) * row[10]) / 10000
                except TypeError:
                    area = 0
                # add area to reDictionary
                try:
                    reDict[re] += area
                except KeyError:
                    continue
                
reDict

# turn into a dataframe and export
df = pd.DataFrame(list(reDict.items()), columns=['re', 'area_ha'])
df.to_csv('Data/regionalEcosystemAreaSummary.csv', index=False)

In [21]:
import os
os.getcwd()

'C:\\Users\\jake.allen.ALLUVIUMQLD\\OneDrive - Alluvium Consulting Australia\\BiodiversityMBRC'